# How to: use LR Schedulers 
1. in `pytorch`
2. in `pytorch-lignting`


Jan 11, 2021

In [6]:
import os,sys
import torch 
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset

In [ ]:
# Select Visible GPU
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

## 1. `torch.optim.lr_scheduler`

In [43]:
# Add learning rate schedulers
from torch.optim.lr_scheduler import ReduceLROnPlateau
model = nn.Linear(1,1)
x = torch.tensor(range(10), dtype=torch.float32).reshape((10,1))
y = 2*x

optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
# Reduce the learning rate of all optimizer param_graphs in `optimizer`
# when the to-be-specified metric/variable's value does not improve ('min' or 'max')
# for `patience`+1 steps. Do the update at `patience`+1 th step since the last lr update
lr_scheduler = ReduceLROnPlateau(optimizer, 'min', 
                                 patience=2, verbose=True)


for ep in range(10):
    print("Start LR: ", optimizer.param_groups[0]['lr'])
    
    # Train step
    out = model(x)
    loss = nn.MSELoss()(out, y)
    
    print(loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Validate loss
    val_loss = model.eval(eval_dl)
    
    # Call lr_scheduler's `step` on the metric based on the val dataset
#     lr_scheduler.step(val_loss)
    lr_scheduler.step(0.1) # For demo purpose, fix to a constance

    print("Updated LR: ", optimizer.param_groups[0]['lr'])
    print("===")
    

Start LR:  0.001
81.50306701660156
Updated LR:  0.001
===
Start LR:  0.001
72.26151275634766
Updated LR:  0.001
===
Start LR:  0.001
64.0684585571289
Updated LR:  0.001
===
Start LR:  0.001
56.8049430847168
Epoch     4: reducing learning rate of group 0 to 1.0000e-04.
Updated LR:  0.0001
===
Start LR:  0.0001
50.365509033203125
Updated LR:  0.0001
===
Start LR:  0.0001
49.77915573120117
Updated LR:  0.0001
===
Start LR:  0.0001
49.199642181396484
Epoch     7: reducing learning rate of group 0 to 1.0000e-05.
Updated LR:  1e-05
===
Start LR:  1e-05
48.62688446044922
Updated LR:  1e-05
===
Start LR:  1e-05
48.57012176513672
Updated LR:  1e-05
===
Start LR:  1e-05
48.513431549072266
Epoch    10: reducing learning rate of group 0 to 1.0000e-06.
Updated LR:  1.0000000000000002e-06
===


## Use `torch.optim.lr_scheduler` in `pytorch-lightning`

In [50]:
import pytorch_lightning as pl
from pytorch_lightning.core.lightning import LightningModule
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning import loggers as pl_loggers

import datetime as dt
from pathlib import Path

ROOT = Path('/data/hayley-old/Tenanbaum2000/')

In [126]:
class LitModule(LightningModule):
    def __init__(self, learning_rate):
        super().__init__()
        self.network = nn.Linear(1,1)
        self.learning_rate = learning_rate
        
    def forward(self, x):
        out = self.network(x)
        return out 
    
    def training_step(self, batch, batch_idx):
        """
        Defines the full forward pass from:
        input --> network --> last node of the computational graph
        """
        x, y = batch
        out = self(x) # calls self.forward (and plus alpha)
        loss = nn.MSELoss()(out, y)
        self.log('train/loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss #or, {"loss": loss}
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        out = self(x) # calls self.forward (and plus alpha)
        loss = nn.MSELoss()(out, y)
        self.log('val/loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return {"val_loss": loss}
    
    def configure_optimizers(self):
        optimizer = optim.SGD(self.parameters(), lr=self.learning_rate)
#         scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,
        lr_scheduler = {
            'scheduler': optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                         mode='min',
                                                         patience=10,
                                                         verbose=True),
            'monitor': 'val_loss',
            'name': "train/lr/mySGD",
        }
        return [optimizer], [lr_scheduler]
#         return (
#             {'optimizer':optimizer, 
#             'lr_scheduler': scheduler,
#             'monitor': 'val_loss',
#             'name': 'network-sgd'}
#         )
    


- Make a train

In [127]:
train_x = torch.tensor(range(10), dtype=torch.float32).reshape((-1,1))
train_y = 2 * train_x
train_ds = TensorDataset(train_x,train_y)
train_dl = DataLoader(train_ds, batch_size=2)

val_x = torch.tensor(range(20), dtype=torch.float32).reshape((-1,1))
val_y = 2 * val_x
val_ds = TensorDataset(val_x, val_y)
val_dl = DataLoader(val_ds, batch_size=20)

In [128]:
x,y = next(iter(train_dl))
x.shape, y.shape

(torch.Size([2, 1]), torch.Size([2, 1]))

- Init. a Lit Module

In [129]:
lr = 1e-2
model = LitModule(learning_rate=lr)

- Init. a trainer with the model and the train data

In [130]:
# Add LR Monitor Callback
cbs = [
    LearningRateMonitor(logging_interval='epoch')
]

# Define logger
today_str = dt.datetime.today().strftime('%Y-%m-%d')
exp_name = f'dump'
tb_logger = pl_loggers.TensorBoardLogger(save_dir=f'{ROOT}/temp-logs/{today_str}', 
                                         name=exp_name,
                                         log_graph=False,
                                        default_hp_metric=False)
print("Log dir: ", tb_logger.log_dir)

log_dir = Path(tb_logger.log_dir)
if not log_dir.exists():
    log_dir.mkdir(parents=True)
    print("Created: ", log_dir)

# Start training
trainer = pl.Trainer(
    logger = tb_logger,
    callbacks= cbs,
    max_epochs=500,
)
trainer.fit(model, train_dl, val_dl)

GPU available: True, used: False
TPU available: None, using: 0 TPU cores

  | Name    | Type   | Params
-----------------------------------
0 | network | Linear | 2     
-----------------------------------
2         Trainable params
0         Non-trainable params
2         Total params


Log dir:  /data/hayley-old/Tenanbaum2000/temp-logs/2021-01-11/dump/version_6
Created:  /data/hayley-old/Tenanbaum2000/temp-logs/2021-01-11/dump/version_6


Epoch   370: reducing learning rate of group 0 to 1.0000e-03.


Epoch   381: reducing learning rate of group 0 to 1.0000e-04.


Epoch   392: reducing learning rate of group 0 to 1.0000e-05.


Epoch   403: reducing learning rate of group 0 to 1.0000e-06.


Epoch   414: reducing learning rate of group 0 to 1.0000e-07.


Epoch   425: reducing learning rate of group 0 to 1.0000e-08.


1

In [113]:
model(x[[5]])

IndexError: index 5 is out of bounds for dimension 0 with size 2

In [114]:
type(model.optimizers())

pytorch_lightning.core.optimizer.LightningSGD

In [115]:
mo = model.optimizers()

In [116]:
mo.param_groups[0]['lr']

1.0000000000000004e-08

In [117]:
model.learning_rate


0.01

In [118]:
trainer.lr_schedulers

[{'scheduler': <torch.optim.lr_scheduler.ReduceLROnPlateau at 0x7f3a48b221c0>,
  'interval': 'epoch',
  'frequency': 1,
  'reduce_on_plateau': True,
  'monitor': 'val_loss',
  'strict': True}]